Library

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import json
import pytesseract
from PIL import Image
import ast
import re
import numpy as np

Fix time

In [58]:
# OCR
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
image = Image.open('data_raw/sb-sprite-uhr-k.png')
my_dict = dict()
for b in range(0, 397, 36):
    for a in range(0, 325, 36):
        text = pytesseract.image_to_string(image.crop((a+5, b+5, a+36-5, b+36-5)), config='--psm 6')
        my_dict.update({f"background-position: -{a}px -{b}px" : text})

with open('data_raw/sb-sprite-uhr-k.json', 'a') as f:
    json.dump(my_dict, f, indent=4)


In [33]:
# update time column
with open('data_raw/sb-sprite-uhr-k.json', 'r') as f:
    my_dict = json.load(f)

with open('data_raw/merge.csv', 'r', encoding='utf-8') as f:
    data = pd.read_csv(f)
    for index, row in data.iterrows():
        row_list = ast.literal_eval(row['time'])
        row_list_1 = row_list[0]
        row_list_2 = row_list[1]
        # print(row_list)
        for i, time in enumerate(row_list_1):
            if row_list_1[i] in my_dict:
                row_list_1[i] = my_dict[row_list_1[i]]
        for i, time in enumerate(row_list_2):
            if row_list_2[i] in my_dict:
                row_list_2[i] = my_dict[row_list_2[i]]
        row_list = [row_list_1, row_list_2]
        data.at[index, 'time'] = row_list

with open('data_raw/output_data_0.csv', 'w', encoding='utf-8') as f:
    data.to_csv(f, index=False, lineterminator='\n')


In [3]:

# Get data from combined_data.csv
with open('data_raw/output_data_filled.csv', 'r', encoding='utf-8') as f:
    data = pd.read_csv(f)

# Convert string elements to lists
data['home_goals'] = data['home_goals'].apply(ast.literal_eval)
data['home_substitution'] = data['home_substitution'].apply(ast.literal_eval)
data['home_card'] = data['home_card'].apply(ast.literal_eval)
data['time'] = data['time'].apply(ast.literal_eval)
data['away_goals'] = data['away_goals'].apply(ast.literal_eval)
data['away_substitution'] = data['away_substitution'].apply(ast.literal_eval)
data['away_card'] = data['away_card'].apply(ast.literal_eval)

# Iterate through each row to add the corresponding time to events
for index, row in data.iterrows():
    j = k = 0
    for i, time in enumerate(row['time'][0]):
        if i < len(row['home_goals']):
            row['home_goals'][i] = [time, row['home_goals'][i]]
        elif j < len(row['home_substitution']):
            row['home_substitution'][j] = [time, row['home_substitution'][j]]
            j += 1
        elif k < len(row['home_card']):
            row['home_card'][k] = [time, row['home_card'][k]]
            k += 1

    j = k = 0
    for i, time in enumerate(row['time'][1]):
        if i < len(row['away_goals']):
            row['away_goals'][i] = [time, row['away_goals'][i]]
        elif j < len(row['away_substitution']):
            row['away_substitution'][j] = [time, row['away_substitution'][j]]
            j += 1
        elif k < len(row['away_card']):
            row['away_card'][k] = [time, row['away_card'][k]]
            k += 1

# # Add ID column
# data['ID'] = range(1, len(data) + 1)

# Save the data just with ID, time, home_goals, home_substitution, home_card, away_goals, away_substitution, away_card
# data = data[['ID', 'time', 'home_goals', 'home_substitution', 'home_card', 'away_goals', 'away_substitution', 'away_card']]
with open('data_raw/output_data_filled.csv', 'w', encoding='utf-8') as f:
    data.to_csv(f, index=False, lineterminator='\n')


5. Processing

In [38]:
# Get starting_lineup from output_data_2.csv
with open('data_raw/output_data_5.csv', 'r', encoding='utf-8') as f:
    data = pd.read_csv(f)

# Handle NaN values and convert string elements (starting_lineup) to lists
data['starting_lineup'] = data['starting_lineup'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

# Iterate through each row to extract and add the new columns
for index, row in data.iterrows():
    if row['starting_lineup']:
        sample_string = row['starting_lineup']
        avg_age = float(re.search(r'Avg\. age: (\d+\.\d+)', sample_string[33]).group(1))
        purchase_value = float(re.search(r'Purchase value: €(\d+\.\d+)', sample_string[33]).group(1))
        total_mv = float(re.search(r'Total MV: €(\d+\.\d+)', sample_string[33]).group(1))
        data.at[index, 'home_avg_age'] = avg_age
        data.at[index, 'home_purchase_value'] = purchase_value
        data.at[index, 'home_total_mv'] = total_mv

# Save the  data just with 4 columns ID, home_avg_age, home_purchase_value, home_total_mv
data = data[['ID', 'home_avg_age', 'home_purchase_value', 'home_total_mv']]
with open('data_raw/output_data_6.csv', 'w', encoding='utf-8') as f:
    data.to_csv(f, index=False, lineterminator='\n')



In [19]:
# Get data from output_data_3.csv
with open('data_raw/output_data_3.csv', 'r', encoding='utf-8') as f:
    data = pd.read_csv(f)

# Remove the first column, year and add the new columns ID at the start, and add year at the end
data = data.drop(columns=['Unnamed: 0'])
data = data.drop(columns=['year'])
data.insert(0, 'ID', range(1, 1 + len(data)))

# Convert string elements (date_time) to lists
data['date_time'] = data['date_time'].apply(ast.literal_eval)
# Create a new column (year)
data['year'] = data['date_time'].apply(lambda x: '20' + str(x[1].split('/')[-1]))

# Save the updated data
with open('data_raw/output_data_4.csv', 'w', encoding='utf-8') as f:
    data.to_csv(f, index=False, lineterminator='\n')


In [ ]:
# Get data from output_data_4.csv
with open('data_raw/output_data_5.csv', 'r', encoding='utf-8') as f:
    data = pd.read_csv(f)

# Convert string elements from stat column to lists
data['stat'] = data['stat'].apply(ast.literal_eval)

# Creat a loop to get the value from stat column and add it to the new columns
for index, row in data.iterrows():
    if row['stat']:
        data.at[index, 'Total_shots_home'] = row['stat'][1].split()[0]
        data.at[index, 'Total_shots_away'] = row['stat'][1].split()[1]
        data.at[index, 'Shots_off_home'] = row['stat'][2].split()[0]
        data.at[index, 'Shots_off_away'] = row['stat'][2].split()[1]
        data.at[index, 'Shots_saved_home'] = row['stat'][3].split()[0]
        data.at[index, 'Shots_saved_away'] = row['stat'][3].split()[1]
        data.at[index, 'Corners_home'] = row['stat'][4].split()[0]
        data.at[index, 'Corners_away'] = row['stat'][4].split()[1]
        data.at[index, 'Free_kicks_home'] = row['stat'][5].split()[0]
        data.at[index, 'Free_kicks_away'] = row['stat'][5].split()[1]
        data.at[index, 'Fouls_home'] = row['stat'][6].split()[0]
        data.at[index, 'Fouls_away'] = row['stat'][6].split()[1]
        data.at[index, 'Off_sides_home'] = row['stat'][7].split()[0]
        data.at[index, 'Off_sides_away'] = row['stat'][7].split()[1]

# Save the  data just with 4 columns ID, Total_shots_home, Total_shots_away, Shots_off_home, Shots_off_away, Shots_saved_home, Shots_saved_away, Corners_home, Corners_shots_away, Free_kicks_home, Free_kicks_away, Fouls_home, Fouls_away, Off_sides_home, Off_sides_away
data = data[['ID', 'Total_shots_home', 'Total_shots_away', 'Shots_off_home', 'Shots_off_away', 'Shots_saved_home', 'Shots_saved_away', 'Corners_home', 'Corners_away', 'Free_kicks_home', 'Free_kicks_away', 'Fouls_home', 'Fouls_away', 'Off_sides_home', 'Off_sides_away']]
with open('data_raw/output_data_7.csv', 'w', encoding='utf-8') as f:
    data.to_csv(f, index=False, lineterminator='\n')
    

In [49]:
# Get data from output_data_5.csv
with open('data_raw/output_data_1.csv', 'r', encoding='utf-8') as f:
    data = pd.read_csv(f)

# Convert string elements to lists
data['home_card'] = data['home_card'].apply(ast.literal_eval)
data['away_card'] = data['away_card'].apply(ast.literal_eval)

# # Show home_card column
# print(data['home_card'][0])
# print(data['home_card'][1])
# print(data['home_card'][14])

# Initialize new columns
data['home_number_of_yellow_card'] = 0
data['home_number_of_red_card'] = 0
data['away_number_of_yellow_card'] = 0
data['away_number_of_red_card'] = 0

# Function to count yellow and red cards
def count_cards(card_list):
    if not card_list or card_list == [[]]:
        return 0, 0
    yellow_cards = sum('Yellow card' in (card[1] or '') for card in card_list if card)
    red_cards = sum('Red card' in (card[1] or '') for card in card_list if card)
    return yellow_cards, red_cards

# Iterate through each row to count the number of yellow and red cards
for index, row in data.iterrows():
    home_yellow, home_red = count_cards(row['home_card'])
    away_yellow, away_red = count_cards(row['away_card'])
    
    data.at[index, 'home_number_of_yellow_card'] = home_yellow
    data.at[index, 'home_number_of_red_card'] = home_red
    data.at[index, 'away_number_of_yellow_card'] = away_yellow
    data.at[index, 'away_number_of_red_card'] = away_red

# Save the updated data
data = data[['ID', 'home_number_of_yellow_card', 'home_number_of_red_card', 'away_number_of_yellow_card', 'away_number_of_red_card']]
with open('data_raw/output_data_9.csv', 'w', encoding='utf-8') as f:
    data.to_csv(f, index=False, lineterminator='\n')

In [3]:
# Count None values in data
with open('data_raw/output_data_5.csv', 'r', encoding='utf-8') as f:
    data = pd.read_csv(f)
    print(data.isnull().sum())



ID                        0
date_time                 0
stadium                   0
home_team                 0
home_position             0
away_team                 0
away_position             0
score                     0
referee                   0
time                      0
home_lineup               0
away_lineup               0
home_squads               0
away_squads               0
home_substitute           0
away_substitute           0
home_goals                0
away_goals                0
home_substitution         0
away_substitution         0
home_card                 0
away_card                 0
home_sanctions            0
away_sanctions            0
starting_lineup           0
substitute               11
manager                  11
stat                      0
url                       0
avg_age                  11
purchase_value           11
total_mv                 11
year                      0
shots_home              248
shots_away              248
shots_on_target_home

In [5]:
import pandas as pd

# Fill missing values based on the previous match information of the same team
def fill_missing_values(data):
    columns_to_fill = ['manager', 'substitute', 'avg_age', 'purchase_value', 'total_mv', 'shots_home', 'shots_away', 'shots_on_target_home', 'shots_on_target_away', 'corners_home', 'corners_away']
    
    for index, row in data.iterrows():
        if any(pd.isnull(row[col]) for col in columns_to_fill):
            home_team = row['home_team']
            away_team = row['away_team']
            
            # Find the previous match where the home_team or away_team is the same
            previous_match = data.loc[(data.index < index) & ((data['home_team'] == home_team) | (data['away_team'] == home_team) | (data['home_team'] == away_team) | (data['away_team'] == away_team))].tail(1)
            
            if not previous_match.empty:
                previous_row = previous_match.iloc[0]
                
                # Fill missing values based on the previous match
                for column in columns_to_fill:
                    if pd.isnull(row[column]):
                        data.at[index, column] = previous_row[column]
    return data

# Apply the function to fill missing values
data = fill_missing_values(data)

# Show the number of missing values
print(data.isnull().sum())

# Save the updated data
with open('data_raw/output_data_filled.csv', 'w', encoding='utf-8') as f:
    data.to_csv(f, index=False, lineterminator='\n')

ID                      0
date_time               0
stadium                 0
home_team               0
home_position           0
away_team               0
away_position           0
score                   0
referee                 0
time                    0
home_lineup             0
away_lineup             0
home_squads             0
away_squads             0
home_substitute         0
away_substitute         0
home_goals              0
away_goals              0
home_substitution       0
away_substitution       0
home_card               0
away_card               0
home_sanctions          0
away_sanctions          0
starting_lineup         0
substitute              0
manager                 0
stat                    0
url                     0
avg_age                 0
purchase_value          0
total_mv                0
year                    0
shots_home              0
shots_away              0
shots_on_target_home    0
shots_on_target_away    0
corners_home            0
corners_away

In [ ]:
# Extract the first row
first_row_df = data.iloc[0]

# Add a new column with information
first_row_df['Information'] = ['ID', 'date_time', 'stadium', 'home_team', 'home_position', 'away_team', 'away_position', 'score', 'referee', 'time', 'home_lineup', 'away_lineup', 'home_squads', 'away_squads', 'home_substitute', 'away_substitute', 'home_goals', 'away_goals', 'home_substitution', 'away_substitution', 'home_card', 'away_card', 'home_sanctions', 'away_sanctions', 'starting_lineup', 'substitute', 'manager', 'stat', 'url', 'avg_age', 'purchase_value', 'total_mv', 'year', 'shots_home', 'shots_away', 'shots_on_target_home', 'shots_on_target_away', 'corners_home', 'corners_away']

# Ensure 'Value' column exists
first_row_df['Value'] = first_row_df.values

# Reorder columns to have 'Information' first
first_row_df = first_row_df[['Information', 'Value']]

# Display the result
print(first_row_df)

# Save the first row to a CSV file
with open('data_raw/first_row.csv', 'w', encoding='utf-8') as f:
    first_row_df.to_csv(f, index=False, lineterminator='\n')

In [22]:
# Load the data from data_train.csv
data_train = pd.read_csv('data_raw/data_train.csv')

# Extract the distinct values of team_id_away and away_team
distinct_values = data_train[['lineup_id_away', 'away_lineup']].drop_duplicates()

# Display the distinct values
print(distinct_values)

distinct_values.to_csv('data_raw/lineupID.csv', index=False)

      lineup_id_away      away_lineup
0                  0          4-2-3-1
2                  1            3-5-2
3                  2   4-4-2 double 6
5                  3          4-1-4-1
11                 4  4-3-3 Attacking
13                 5            4-4-2
15                 6          4-4-1-1
17                 7       3-5-2 flat
18                 8            5-4-1
20                 9          3-1-4-2
22                10  4-3-3 Defending
33                11    4-4-2 Diamond
34                12          3-4-2-1
36                13          4-3-1-2
70                14       4-5-1 flat
105               15            5-3-2
237               16    5-4-1 Diamond
442               17          4-3-2-1
786               18            3-4-3
914               19            4-5-1
971               20          3-4-1-2
1568              21          4-1-3-2
2605              22  3-5-2 Attacking
2632              23          3-3-3-1


In [7]:
# Load the timeID.csv file
time_id_df = pd.read_csv('data_raw/timeID.csv')
team_data = pd.read_csv('data_raw/team_data.csv')

# Create a dictionary to map team names to their IDs
team_name_to_id = dict(zip(time_id_df['team_name'], time_id_df['team_id']))

# Map the team names to their IDs in the data_train DataFrame
team_data['team_id'] = team_data['team_id'].map(team_name_to_id)

# Save the updated data_train DataFrame to a new CSV file
team_data.to_csv('data_raw/team_data.csv', index=False)

In [ ]:
team_data = pd.read_csv('data_raw/team_data.csv')
# Convert string elements (date_time) to lists
team_data['date_time'] = team_data['date_time'].apply(ast.literal_eval)

# Extract year, month, and day from date_time
team_data['year'] = team_data['date_time'].apply(lambda x: '20' + str(x[1].split('/')[-1].split(' ')[0]))
team_data['month'] = team_data['date_time'].apply(lambda x: str(x[1].split('/')[0].split(' ')[-1]))
team_data['day'] = team_data['date_time'].apply(lambda x: str(x[1].split('/')[1]))

# Save the updated data
with open('data_raw/team_data.csv', 'w', encoding='utf-8') as f:
    team_data.to_csv(f, index=False, lineterminator='\n')

In [15]:
# Remove rows with null values in team_data
team_data = team_data.dropna()

# Save the updated data
with open('data_raw/team_data.csv', 'w', encoding='utf-8') as f:
    team_data.to_csv(f, index=False, lineterminator='\n')

In [16]:
# Drop the date_time column
# team_data = team_data.drop(columns=['date_time'])
# Convert all numeric columns to integers
numeric_columns = team_data.select_dtypes(include=['float64']).columns
team_data[numeric_columns] = team_data[numeric_columns].astype(int)

# Save the updated data
with open('data_raw/team_data.csv', 'w', encoding='utf-8') as f:
    team_data.to_csv(f, index=False, lineterminator='\n')


In [19]:
# Load the team_data.csv file
team_data = pd.read_csv('data_raw/team_data.csv')

# Filter the data for team_id = 2
team_data_filtered = team_data[team_data['team_id'] == 2]

# Display the filtered data
print(team_data_filtered)

# Save the filtered data to a new CSV file
team_data_filtered.to_csv('data_raw/team_data_filtered.csv', index=False)

# Sort the data based on year, month, and day
team_data_filtered = team_data_filtered.sort_values(by=['year', 'month', 'day'])

# Add an index column
team_data_filtered.reset_index(drop=True, inplace=True)
team_data_filtered['index'] = team_data_filtered.index

# Display the sorted and indexed data
print(team_data_filtered)

# Save the sorted and indexed data to a new CSV file
team_data_filtered.to_csv('data_raw/team_data_filtered_sorted.csv', index=False)

      team_id  Total_shots  Shots_off  Shots_saved  Corners  Free_kicks  \
5           2            6          4            2        4          11   
14          2            5          5            1        4          21   
41          2            1          3            3        6           9   
70          2            9          4            0        3           9   
97          2            7          3            0        4          15   
...       ...          ...        ...          ...      ...         ...   
6968        2           17          5            5        9           6   
6993        2            7          2            6        2          12   
7006        2           27          8            6        6          10   
7018        2           14          6            5        4           6   
7041        2           11          2            2        5          10   

      Off_sides  Fouls  year  month  day  
5             1     15  2014      8   24  
14           